# **DESAFIO 01 - EVASÃO BANCÁRIA (CHURN)**

Vamos usar [este conjunto de dados](https://github.com/cassiusf/datasets/raw/refs/heads/main/desafios/Churn_Dataset.csv) para tentar prever evasão de clientes no setor bancário.

Este conjunto de dados contém informações sobre clientes bancários que deixaram o banco ou continuam sendo seus clientes. O conjunto de dados inclui as seguintes variáveis:

- **Customer ID**: identificador único para cada cliente
- **Surname**: sobrenome do cliente
- **Credit Score**: valor numérico representando o score de crédito do cliente, ou seja, uma classificação interna do quão bom pagador ele é
- **Geography**: país onde o cliente reside (França, Espanha ou Alemanha)
- **Gender**: gênero do cliente (masculino ou feminino)
- **Age**: idade do cliente.
- **Tenure**: número de anos que o cliente está no banco.
- **Balance**: saldo da conta do cliente.
- **NumOfProducts**: número de produtos bancários utilizados pelo cliente (por exemplo, conta poupança, cartão de crédito).
- **HasCrCard**: se o cliente tem um cartão de crédito (1 = sim, 0 = não).
- **IsActiveMember**: se o cliente é um membro ativo (1 = sim, 0 = não).
- **EstimatedSalary**: renda estimada do cliente.
- **Exited**: evasão do cliente (1 = sim, 0 = não) ***(Variável alvo)***.


## **TAREFA**

Seu objetivo é utilizar um modelo de machine learning rasa para prever a possibilidade de um cliente cancelar sua conta no banco.

Você deverá fazer todas es etapas necessárias tanto na avaliação dos dados disponíveis quanto na preparação necessária para execução do modelo escolhido.

Todas as etapas devem ser documentadas e as escolhas justificadas.

A performance do seu modelo deverá ser avaliada pelas métricas Acurácia, Precision, Recall e F1-Score. Inclua em sua avaliação final de performance Curvas, ROC e/ou Precision-Recall, de acordo com a necessidade.

In [20]:
import pandas as pd
import matplotlib.pyplot as plt
# Importa o conjunto de dados
df = pd.read_csv("https://raw.githubusercontent.com/cassiusf/datasets/refs/heads/main/desafios/Churn_Dataset.csv")
pd.reset_option('all')




/tmp/ipykernel_3717/1931712287.py:5: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  pd.reset_option('all')


In [21]:
pd.set_option('display.max_columns', None)
df

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42.0,2,0.00,1,1.0,1.0,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41.0,1,83807.86,1,0.0,1.0,112542.58,0
2,3,15619304,Onio,502,France,Female,42.0,8,159660.80,3,1.0,0.0,113931.57,1
3,4,15701354,Boni,699,France,Female,39.0,1,0.00,2,0.0,0.0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43.0,2,125510.82,1,NaN,1.0,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9997,9998,15584532,Liu,709,France,Female,36.0,7,0.00,1,0.0,1.0,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42.0,3,75075.31,2,1.0,0.0,92888.52,1
9999,9999,15682355,Sabbatini,772,Germany,Male,42.0,3,75075.31,2,1.0,0.0,92888.52,1
10000,10000,15628319,Walker,792,France,Female,28.0,4,130142.79,1,1.0,0.0,38190.78,0


In [22]:
##verifiquei quantos nulos exitem

df.isna().sum()



RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          1
Gender             0
Age                1
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          1
IsActiveMember     1
EstimatedSalary    0
Exited             0
dtype: int64

In [23]:
##decide excluir os nulos

df= df.dropna()




In [ ]:
##removendo colunas irrelevantes 

from sklearn.preprocessing import OneHotEncoder, StandardScaler

df.drop(columns=['CustomerId', 'Surname'], inplace=True)

In [28]:
##codificando variveis categoricas 
df = pd.get_dummies(df,columns=["Geography","Gender"], drop_first=True)


In [30]:
##normalização variaveis numericas 
scaler = StandardScaler()
num_cols = ['CreditScore', 'Age', 'Balance', 'EstimatedSalary']
df[num_cols] = scaler.fit_transform(df[num_cols])

In [ ]:
##criei uma tabela so de number para verificar  outlier
df_numerico = df.select_dtypes(include=['int64', 'float64'])
df_numerico = df_numerico.drop(columns=(['Exited',"IsActiveMember","HasCrCard","RowNumber","CustomerId"]))
df_numerico



In [ ]:
##fazendo boxplot para verificar os outlier de cada coluna
for coluna in df_numerico.columns:
    plt.figure(figsize=(5, 3))
    plt.boxplot(df_numerico[coluna])
    plt.title(f'Boxplot de {coluna}')
    plt.ylabel('Valores')
    plt.show()
    

In [32]:
##sepração de treino e teste 

from sklearn.model_selection import train_test_split
##tirando variavel alvo
y = df["Exited"]
X = df.drop(columns=["Exited"])

#separando teste e treino 

X_train , X_test, y_train, y_test =  train_test_split(X,y,test_size=0.3 )